### Chapter 29: Manipulating DataFrames

Typically once you have a data frame, you will need to manipulate it.  This includes filtering (subsets) rows or columns, creating new columns.  We will cover that in this section.   We'll also use the following simple dataset for examples:



In [3]:
using DataFrames, Chain, Statistics

In [4]:
data = DataFrame(A = 1:2:13, B = ["M", "F", "F", "M","X","F","M"], C=[3.0,2.5,pi,-2.3,1/3,56,100],D=[(-1)^n//n for n=1:7])

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,1,M,3.0,-1//1
2,3,F,2.5,1//2
3,5,F,3.14159,-1//3
4,7,M,-2.3,1//4
5,9,X,0.333333,-1//5
6,11,F,56.0,1//6
7,13,M,100.0,-1//7




#### 29.1: Subsetting Dataframes

We first start with finding a subset of a DataFrame.  This can be either a smaller number of rows based on some condition or a subset of the columns.

First, if we want to select specific columns, we can do this in a couple of different ways.  One way is similar to array access like the previous chapter.  The following selects the A and D column.  Note the colon (:) in front of the column names.  

In [5]:
data[!,[:A,:D]]

Row,A,D
,Int64,Rational…
1,1,-1//1
2,3,1//2
3,5,-1//3
4,7,1//4
5,9,-1//5
6,11,1//6
7,13,-1//7


Alternatively, we can use the `select` function.  The first argument is the data frame and the others are columns names.

In [6]:
select(data, :A, :D)

Row,A,D
,Int64,Rational…
1,1,-1//1
2,3,1//2
3,5,-1//3
4,7,1//4
5,9,-1//5
6,11,1//6
7,13,-1//7


And we can use column numbers and reorder:

In [7]:
select(data, 4,3)

Row,D,C
,Rational…,Float64
1,-1//1,3.0
2,1//2,2.5
3,-1//3,3.14159
4,1//4,-2.3
5,-1//5,0.333333
6,1//6,56.0
7,-1//7,100.0


### Using a @chain block

Although this is just a single line, we'll start using a `@chain` block to do calculations and we'll stay with this style for the rest of this chapter:

#### Not and Between

A couple of helpful select options are the `Not` and `Between` functions. with `select`, this returns either columns that are not the given one or those between two columns:

In [8]:
@chain data begin
  select(Not(:B))
end

Row,A,C,D
,Int64,Float64,Rational…
1,1,3.0,-1//1
2,3,2.5,1//2
3,5,3.14159,-1//3
4,7,-2.3,1//4
5,9,0.333333,-1//5
6,11,56.0,1//6
7,13,100.0,-1//7


Let's find all columns bewtween B and D

#### Filtering (or subsetting) the rows

Next, we see how to filter (or subset) the rows based on some condition.  This example shows that we take only the rows where the column A values are less than 10.

In [11]:
@chain data begin
  subset(:A => a-> a .< 10)
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,1,M,3.0,-1//1
2,3,F,2.5,1//2
3,5,F,3.14159,-1//3
4,7,M,-2.3,1//4
5,9,X,0.333333,-1//5


Note that the last argument is a function (anonymous) whose input is the entire column and we want a vector of booleans.  This is why the less than sign is broadcast (`.<`).

Alternatively, we can use the `ByRow` function on a non-vector function like:

In [11]:
@chain data begin
  subset(:A => ByRow(a-> a < 10))
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,1,M,3.0,-1//1
2,3,F,2.5,1//2
3,5,F,3.14159,-1//3
4,7,M,-2.3,1//4
5,9,X,0.333333,-1//5


This filters all rows where column B is "F":

In [ ]:
@chain data begin
  subset(:B => b-> b.== "F")
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,3,F,2.5,1//2
2,5,F,3.14159,-1//3
3,11,F,56.0,1//6


We can filter on more than one column.  This example returns all rows where column A is larger than column C.

Note: that the columns must be put into vector format and then the function must have 2 columns. 

In [13]:
@chain data begin
  subset([:A, :C] => (a,c) -> a .> c)
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,3,F,2.5,1//2
2,5,F,3.14159,-1//3
3,7,M,-2.3,1//4
4,9,X,0.333333,-1//5


And here's an example using three columns

In [14]:
@chain data begin
  subset([:A, :D, :C] => (a,d,c) -> a.*d .> c)
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,7,M,-2.3,1//4


#### Exercise

- find all rows where the absolute value of the C column is greater than 2.
- find all rows where the product of columns C and D is greater than 1.

### 29.3: Transforming Data Frames

If you want a new column that is some function of one or more of the columns, we will use either `select` or `transform`:
- use `select` if you only want the new column(s) in the data frame
- use `transform` if you want the original data frame as well as the new columns

The following makes a dataframe with a single column that is the square of the A column

In [15]:
@chain data begin
  select(:A => a-> a.^2)
end

Row,A_function
,Int64
1,1
2,9
3,25
4,49
5,81
6,121
7,169


Notice that the new column has the generic column `A_function`. Instead, if we want to give that column a better name use: 

In [16]:
@chain data begin
  select(:A => (a-> a.^2) => :Asq)
end

Row,Asq
,Int64
1,1
2,9
3,25
4,49
5,81
6,121
7,169


Note: make sure the ( ) are around the function.  Remove them to see what happens.

We can also make a column based on a function of two columns. For example:

In [17]:
@chain data begin
  select([:C, :D] => ((c,d)-> c.*d) => :prod)
end

Row,prod
,Float64
1,-3.0
2,1.25
3,-1.0472
4,-0.575
5,-0.0666667
6,9.33333
7,-14.2857


And if we want to do both:

In [18]:
@chain data begin
  select(:A => (a-> a.^2) => :Asq, [:C, :D] => ((c,d)-> c.*d) => :prod)
end

Row,Asq,prod
,Int64,Float64
1,1,-3.0
2,9,1.25
3,25,-1.0472
4,49,-0.575
5,81,-0.0666667
6,121,9.33333
7,169,-14.2857


When we use `select`, only the new columns are returned.  If instead we want the original dataframe along with the new ones, we can add additional columns to it with the `transform` function

In [19]:
@chain data begin
  transform(:A => (a-> a.^2) => :Asq)
end

Row,A,B,C,D,Asq
,Int64,String,Float64,Rational…,Int64
1,1,M,3.0,-1//1,1
2,3,F,2.5,1//2,9
3,5,F,3.14159,-1//3,25
4,7,M,-2.3,1//4,49
5,9,X,0.333333,-1//5,81
6,11,F,56.0,1//6,121
7,13,M,100.0,-1//7,169


#### Exercise

- create a new data frame from `data` which is the square root of column C.  Use both `select` and `transform` to see the difference.
- Using the `iris` dataframe produce a new column called `area` which is the area of a petal using the `PetalLength` and `PetalWidth` variables and the area of an ellipse. Keep the original columns with this new dataframe.

### 29.4: Sorting DataFrames

Sorting data frames is quite helpful in many situations.  We use the `sort` function to do this.  The following sorts on column C

In [12]:
@chain data begin
  sort(:C)
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,7,M,-2.3,1//4
2,9,X,0.333333,-1//5
3,3,F,2.5,1//2
4,1,M,3.0,-1//1
5,5,F,3.14159,-1//3
6,11,F,56.0,1//6
7,13,M,100.0,-1//7


And if we want to sort in reverse order

In [24]:
@chain data begin
  sort(:C, rev = true)
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,13,M,100.0,-1//7
2,11,F,56.0,1//6
3,5,F,3.14159,-1//3
4,1,M,3.0,-1//1
5,3,F,2.5,1//2
6,9,X,0.333333,-1//5
7,7,M,-2.3,1//4


And sorting is done by type.  This sorts lexiographically.

In [23]:
@chain data begin
  sort(:B)
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,3,F,2.5,1//2
2,5,F,3.14159,-1//3
3,11,F,56.0,1//6
4,1,M,3.0,-1//1
5,7,M,-2.3,1//4
6,13,M,100.0,-1//7
7,9,X,0.333333,-1//5


If there are ties in sorting (see above), we can sort by a second column as in the following

In [25]:
@chain data begin
  sort([:B, :D])
end

Row,A,B,C,D
,Int64,String,Float64,Rational…
1,5,F,3.14159,-1//3
2,11,F,56.0,1//6
3,3,F,2.5,1//2
4,1,M,3.0,-1//1
5,13,M,100.0,-1//7
6,7,M,-2.3,1//4
7,9,X,0.333333,-1//5


And notice that within all rows where column B is "F", that the rows are sorted by column D.

### 29.5: Joining DataFrames

Another important activity to do with data frames is joining two or more.  Typically this means that both data frame have a common piece of information on which to join.  Consider the following:

In [13]:
simpsons = DataFrame(
  name=["Homer","Marge","Lisa","Bart","Maggie","Apu","Moe", "Milhouse", "Patty"],
  age =[45,42,8,10,1,38,59, 1, 46],
  current_school_grade = [missing, missing, 2, 4, missing, missing, missing, 4, missing],
  favorite_food = ["pork chops","casserole","salad","hamburger",missing,"saag paneer","peanuts", missing, "Lady Laramie 100s"]
)

Row,name,age,current_school_grade,favorite_food
,String,Int64,Int64?,String?
1,Homer,45,missing,pork chops
2,Marge,42,missing,casserole
3,Lisa,8,2,salad
4,Bart,10,4,hamburger
5,Maggie,1,missing,missing
6,Apu,38,missing,saag paneer
7,Moe,59,missing,peanuts
8,Milhouse,1,4,missing
9,Patty,46,missing,Lady Laramie 100s


In [14]:
jobs = DataFrame(
  name = ["Homer","Marge","Apu","Moe", "Patty", "Wiggam"],
  job = ["nuclear technician", "housewife", "store owner", "bartender", "DMV clerk", "police chief"],
  salary = [50_000, 25_000, 60_000, 15_000, missing, 75_000]
)

Row,name,job,salary
,String,String,Int64?
1,Homer,nuclear technician,50000
2,Marge,housewife,25000
3,Apu,store owner,60000
4,Moe,bartender,15000
5,Patty,DMV clerk,missing
6,Wiggam,police chief,75000


If we want to join these two dataframe, this means add columns to all or some of the rows.  We need to join them on one of the columns from each. For this, we will join on the `Name` column from both. 

In [30]:
innerjoin(simpsons, jobs, on = :name)

Row,name,age,current_school_grade,favorite_food,job,salary
,String,Int64,Int64?,String?,String,Int64?
1,Homer,45,missing,pork chops,nuclear technician,50000
2,Marge,42,missing,casserole,housewife,25000
3,Apu,38,missing,saag paneer,store owner,60000
4,Moe,59,missing,peanuts,bartender,15000
5,Patty,46,missing,Lady Laramie 100s,DMV clerk,missing


First, the first 4 columns came from `simpsons` and the last 2 from `jobs`.   Note: if you want to join on two different columns, like `col1` from the first dataframe and `col2` from the second, use `on = :col1 => :col2` in the join. 

There are many different kinds of joins between dataframes and the difference depends on which rows are kept.  `innerjoin` keeps only rows in which the joining column exists in both dataframes.  Notice that the row with Milhouse is only in the first dataframe and Wiggam is only in the second.  Neither of these are in the resulting dataframe. 

Another join is the `outerjoin` which results in

In [31]:
outerjoin(simpsons, jobs, on = :name)

Row,name,age,current_school_grade,favorite_food,job,salary
,String,Int64?,Int64?,String?,String?,Int64?
1,Homer,45,missing,pork chops,nuclear technician,50000
2,Marge,42,missing,casserole,housewife,25000
3,Apu,38,missing,saag paneer,store owner,60000
4,Moe,59,missing,peanuts,bartender,15000
5,Patty,46,missing,Lady Laramie 100s,DMV clerk,missing
6,Lisa,8,2,salad,missing,missing
7,Bart,10,4,hamburger,missing,missing
8,Maggie,1,missing,missing,missing,missing
9,Milhouse,1,4,missing,missing,missing


And notice that this keeps all rows from both dataframes.  It also inserts `missing` anywhere where a row comes from a dataframe that is only in one of the two.  Wiggam has missing data from the columsn 2-4 and Maggie and Bart are missing the last columns. 

#### Exercise

1. Perform a `leftjoin` on these two dataframes.
2. Perform a `rightjoin` on these two dataframes.
3. What do each of them do. 

The following are the joins in the `DataFrames` package:

- **innerjoin:** the output contains rows for values of the key that exist in all passed data frames.
- **leftjoin:** the output contains rows for values of the key that exist in the first (left) argument, whether or not that value exists in the second (right) argument.
- **rightjoin:** the output contains rows for values of the key that exist in the second (right) argument, whether or not that value exists in the first (left) argument.
- **outerjoin:** the output contains rows for values of the key that exist in any of the passed data frames.
- **semijoin:** Like an inner join, but output is restricted to columns from the first (left) argument.
- **antijoin:** The output contains rows for values of the key that exist in the first (left) but not the second (right) argument. As with semijoin, output is restricted to columns from the first (left) argument.
- **crossjoin:** The output is the cartesian product of rows from all passed data frames.

## 29.6: Summarizing Data

Usually one wants to boil down a dataset to a few numbers. This is typically what the mean, median, standard deviation and quartiles are useful for.  

The `combine` function in `DataFrames` will do this:

In [15]:
combine(data, :A => mean, :C => sum, :C => std, :D => (d -> quantile(d,0.75)) )

Row,A_mean,C_sum,C_std,D_function
,Float64,Float64,Float64,Float64
1,7.0,162.675,39.5518,0.208333


where the last function returns the 3rd quartile (0.75 quantile). 

We will see `combine` below in which a dataset is grouped and then computations are made on a per group basis.

### 29.7: Performing multiple steps on a Dataframe

Although this is perhaps a bit of a hokie example, let's say that we want to do the following steps:

1. Merge/join the simpsons and jobs DataFrames as shown above.
2. Make a new column that is the salary column minus 400 times the age. Call it happiness.
3. Drop any rows in which the happiness is missing.
4. Filter out any rows with the happiness column is negative.
5. Include only the columns name, age, favorite_food and happiness
6. Sort the result by happiness with largest values on top.

To do this, we will put all of these steps in a `@chain` block as follows:

In [ ]:
@chain simpsons begin

end

Row,name,age,current_school_grade,favorite_food
,String,Int64,Int64?,String?
1,Homer,45,missing,pork chops
2,Marge,42,missing,casserole
3,Lisa,8,2,salad
4,Bart,10,4,hamburger
5,Maggie,1,missing,missing
6,Apu,38,missing,saag paneer
7,Moe,59,missing,peanuts
8,Milhouse,1,4,missing
9,Patty,46,missing,Lady Laramie 100s


###  29.8: Split-Apply-Combine

A common situation with data analysis is to have a dataset and you want to compare means or standard deviations within a dataset. What needs to often happen is that you first split a dataset, do some analysis on each group then summarize. This is know as *split-apply-combine*. We will demonstrate this with an example. Let’s return to the iris dataset that we loaded at the beginning of this chapter.

First, let load the dataset again with: 

In [41]:
using RDatasets
ENV["DATAFRAMES_ROWS"] = 10
iris = RDatasets.dataset("datasets", "iris")

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
⋮,⋮,⋮,⋮,⋮,⋮
146,6.7,3.0,5.2,2.3,virginica
147,6.3,2.5,5.0,1.9,virginica
148,6.5,3.0,5.2,2.0,virginica


The following splits the `iris` dataset by the `Species` column. (there are 3)

In [42]:
gdf = groupby(iris, :Species)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
⋮,⋮,⋮,⋮,⋮,⋮
46,4.8,3.0,1.4,0.3,setosa
47,5.1,3.8,1.6,0.2,setosa
48,4.6,3.2,1.4,0.2,setosa


Notice that you see the first and last groups and it is announced that there are 3 groups.

The `combine` function will summarize and result in a row per group.  Let's say we want the mean and standard deviation of the `PetalLength` variable:

In [43]:
combine(gdf, :PetalLength => mean, :PetalLength => std)

Row,Species,PetalLength_mean,PetalLength_std
,Cat…,Float64,Float64
1,setosa,1.462,0.173664
2,versicolor,4.26,0.469911
3,virginica,5.552,0.551895


If we just want the number of rows in each group

In [44]:
combine(gdf, nrow)

Row,Species,nrow
,Cat…,Int64
1,setosa,50
2,versicolor,50
3,virginica,50


Typically, there are multiple steps involved in the split-apply-combine.  Let's say that we want to split the iris data set as above, find the maximum of the `SepalLength`, the median of the `SepalWidth` and then the mean area of the petal (as found above in the exercise)

In [ ]:
@chain iris begin
  transform([:PetalWidth, :PetalLength] => ((w,l) -> pi*w.*l) => :PetalArea)
  groupby(:Species)
  combine(:SepalLength => maximum, :SepalWidth => median, :PetalArea => mean)
end

Row,Species,SepalLength_maximum,SepalWidth_median,PetalArea_mean
,Cat…,Float64,Float64,Float64
1,setosa,5.8,3.4,1.14857
2,versicolor,7.0,2.8,17.9712
3,virginica,7.9,3.0,35.4881
